In [1]:
cd ..

C:\Users\Agustin\Dropbox\Doctorado\NeuroData\PyLeech


In [2]:
cd ..

C:\Users\Agustin\Dropbox\Doctorado\NeuroData


In [3]:
%matplotlib qt5

In [4]:
import PyLeech.Utils.AbfExtension as abfe
import PyLeech.Utils.CrawlingDatabaseUtils as CDU
from PyLeech.Utils.burstStorerLoader import UnitInfo
import PyLeech.Utils.burstUtils as burstUtils
import numpy as np
import matplotlib.pyplot as plt
import PyLeech.Utils.NLDUtils as NLD
import scipy.signal as spsig
import os.path
import winsound
from mpl_toolkits.mplot3d import Axes3D

In [5]:
cdd = CDU.loadDataDict()
file_list = list(cdd)
trace_list = []
emb_list = []
ran_files = []
run_dict = {}

In [8]:
fn = file_list[2]
print(fn)
print(list(cdd[fn]['neurons']))

RegistrosDP_PP\2018_12_03_0005.pklspikes
[3, 1, 2, 6]


In [9]:
a = [print(i, fn) for fn, i in zip(file_list, range(len(file_list)))]

0 RegistrosDP_PP\18n05010.pklspikes
1 RegistrosDP_PP\2018_10_11_0002.pklspikes
2 RegistrosDP_PP\2018_12_03_0005.pklspikes
3 RegistrosDP_PP\2018_12_04_0003_0004_0005_0006b.pklspikes
4 RegistrosDP_PP\2018_12_06_0005.pklspikes
5 RegistrosDP_PP\2018_12_13_0015.pklspikes
6 RegistrosDP_PP\NS_DP_PP_0.pklspikes
7 RegistrosDP_PP\NS_T_DP_PP_0_cut.pklspikes
8 RegistrosDP_PP\NS_T_DP_PP_1.pklspikes
9 RegistrosDP_PP\2014_09_25_0008.pklspikes
10 RegistrosDP_PP\2019_01_28_0001.pklspikes
11 RegistrosDP_PP\2018_11_06_0004.pklspikes
12 RegistrosDP_PP\2018_12_03_0000_0001.pklspikes
13 RegistrosDP_PP\14217000.pklspikes
14 RegistrosDP_PP\2017_12_08_0002.pklspikes
15 RegistrosDP_PP\cont10.pklspikes
16 RegistrosDP_PP\2018_12_13_0001.pklspikes


In [14]:
fn = file_list[4]
print(fn)

RegistrosDP_PP\2018_12_06_0005.pklspikes


In [15]:
burst_obj = UnitInfo(fn, 'RegistrosDP_PP', mode='load')
arr_dict, time_vector1, fs = abfe.getArraysFromAbfFiles(fn, ['Vm1'])

binning_dt = 0.25
spike_kernel_sigma = 1
kernel = NLD.generateGaussianKernel(sigma=spike_kernel_sigma, time_range=20, dt_step=binning_dt)

conv_NS = spsig.fftconvolve(arr_dict["Vm1"], kernel, mode='same')[::int(binning_dt*fs)]
del arr_dict

In [16]:
burst_object = UnitInfo(fn, 'RegistrosDP_PP', 'load')
burst_object.isDe3 = cdd[fn]["DE3"]

good_neurons = [neuron for neuron, neuron_dict in cdd[fn]['neurons'].items() if neuron_dict['neuron_is_good']]

spike_freq_array = burstUtils.processSpikeFreqDict(burst_object.spike_freq_dict, binning_dt,
                                                   selected_neurons=good_neurons,
                                                   time_length=burst_object.time[-1])



smoothed_sfd = {}
for key, items in spike_freq_array.items():
    smoothed_sfd[key] = np.array([items[0], spsig.fftconvolve(items[1], kernel, mode='same')])
conv_NS[(conv_NS > -36) | (conv_NS < -65)] = np.nan


In [17]:
fig, ax_list = burstUtils.plotFreq(burst_object.spike_freq_dict, color_dict='single_color', draw_list=good_neurons, scatter_plot=True, optional_trace=[time_vector1[::int(binning_dt*fs)], conv_NS], outlier_thres=3.5)
burstUtils.plotFreq(spike_freq_array, color_dict='single_color', draw_list=good_neurons, scatter_plot=True, optional_trace=[time_vector1[::int(binning_dt*fs)], conv_NS], sharex=ax_list[0])
burstUtils.plotFreq(smoothed_sfd, color_dict='single_color', draw_list=good_neurons, scatter_plot=False, optional_trace=[time_vector1[::int(binning_dt*fs)], conv_NS], sharex=ax_list[0])
spike_idxs = NLD.getSpikeIdxs(smoothed_sfd, cdd[fn]["DE3"], cdd[fn]["crawling_intervals"])

In [18]:
### para file 4: 5, 10, 12, 21
### para file 5: 3, 21, 8, 13
phase_1 = smoothed_sfd[5][1]/smoothed_sfd[5][1].max()
phase_2 = smoothed_sfd[10][1]/smoothed_sfd[10][1].max()
antiphase_1 = smoothed_sfd[12][1]/smoothed_sfd[12][1].max()
antiphase_2 = smoothed_sfd[21][1]/smoothed_sfd[21][1].max()

In [19]:
summed_activity = antiphase_1 + antiphase_2 - phase_1 - phase_2

In [20]:
fig, ax = plt.subplots(3, 1, sharex=True)
ax[0].plot(phase_1, label='ph1')
ax[0].plot(phase_2, label='ph2')
ax[0].plot(antiphase_1, label='antiph1')
ax[0].plot(antiphase_2, label='antiph2')
ax[1].plot(summed_activity)
ax[2].plot(conv_NS)
fig.legend()

In [16]:
sc_ns = (conv_NS[3000:7000]-np.median(conv_NS[3000:7000]))/(64-46)
fig, ax = plt.subplots()
ax.plot(-sc_ns)
ax.plot(summed_activity[3000:7000])

In [17]:
summed_activity = summed_activity[2860:8800]
embedding = NLD.getDerivativeEmbedding(summed_activity, dt=.1, emb_size=3)

In [18]:
emb_rt = NLD.getCloseReturns(embedding, get_mask=True, threshold=.05)
trace_rt = NLD.getCloseReturns(np.array([summed_activity]).T, get_mask=True, threshold=.05)

In [19]:
data1 = NLD.plotCloseReturns(emb_rt, reorder=False)
data2 = NLD.plotCloseReturns(trace_rt, reorder=False)

In [20]:
emb_sg_dict = NLD.getCloseReturnsSegmentsFromUnorderedMatrix1(emb_rt, single_file=True, min_dist=50, rt_len=150)
trace_sg_dict = NLD.getCloseReturnsSegmentsFromUnorderedMatrix1(trace_rt, single_file=True, min_dist=50, rt_len=150)

In [21]:
h = 0 
for diag, pairs in emb_sg_dict.items():
    h += len(pairs)
k = 0
for diag, pairs in trace_sg_dict.items():
    k += len(pairs)
print(h, k)

53 94


In [25]:
# for diag in NLD.getMainDiagonals(list(emb_sg_dict)):
# for diag in list(emb_sg_dict):
#     for t0, t1 in emb_sg_dict[diag]:
# for diag in list(emb_sg_dict):
for diag in dl:
    for t0, t1 in emb_sg_dict[diag]:
        fig, ax = plt.subplots()
        ax.plot(summed_activity[t0:t1])
        ax.plot(summed_activity[t0+diag:t1+diag])
        fig.suptitle(str(diag))
        
# for diag in list(trace_sg_dict):
#     for t0, t1 in trace_sg_dict[diag]:
#         fig, ax = plt.subplots()
#         ax.plot(summed_activity[t0:t1])
#         ax.plot(summed_activity[t0+diag:t1+diag])
#         fig.suptitle(str(diag))

In [32]:
dl = [112, 175, 286, 486, 684, 831]
for diag in dl:
    for t0, t1 in emb_sg_dict[diag]:
        fig = plt.figure()
        ax = Axes3D(fig)
        NLD.plot3Dline(embedding[t0:t1], fig_ax_pair=[fig, ax])
        NLD.plot3Dline(embedding[t0+diag:t1+diag], fig_ax_pair=[fig, ax])

In [35]:
pos = spsig.find_peaks(summed_activity, height=.4)[0]

In [37]:
fig, ax = plt.subplots()
ax.plot(summed_activity)
for i in pos:
    ax.scatter(i, summed_activity[i], color='r')

In [40]:
sg = []
for i in range(pos.shape[0]-1):
    print(summed_activity[pos[i]:pos[i+1]].shape)
    sg.append(summed_activity[pos[i]:pos[i+1]])

(175,)
(176,)
(234,)
(252,)
(191,)
(143,)
(206,)
(121,)
(119,)
(157,)
(126,)
(177,)
(90,)
(138,)
(115,)
(149,)
(159,)
(100,)
(141,)
(109,)
(121,)
(89,)
(61,)
(131,)
(93,)
(408,)
(422,)
(168,)
(608,)
(66,)
(48,)
(86,)
(41,)


In [46]:
fig, ax = plt.subplots()
for segment in sg:
    if segment.shape[0]<300:
        ax.plot(segment)